This code will show how to generate basic reports and then carry out clean up and taxonomic classification of paired-end metagenome .fastq.gz files in BASH. 

Sample data from ENA:

Arctic Ocean metagenomes sampled aboard CGC Healy during the 2015 GEOTRACES Arctic research cruise Secondary Study Accession:ERP015773 Study Title:Arctic Ocean metagenomes from HLY1502 Center Name:UNIVERSITY OF ALASKA FAIRBANKS Study Name:Arctic Ocean metagenomes ENA-FIRST-PUBLIC:2016-05-27 ENA-LAST-UPDATE:2016-05-25

Can be found at: https://www.ebi.ac.uk/ena/browser/view/PRJEB14154?show=reads

I have used the first 5 pairs of Generated FASTQ files

move into content folder:

In [1]:
cd example_content

examine content...notice all of the .fastq.gz files are in separate subfolders

In [2]:
ls

ERR1424899	ERR1424900	ERR1424901	ERR1424902	ERR1424903


move up one directory and create a new subdirectory to move all of the .fastq.gz files into one place. Then check that the directory was made with ls.

In [3]:
cd ..

First, we need to make a copy of the original data before moving it

In [4]:
cp -R example_content example_content_copy

In [5]:
mkdir all_data

In [6]:
ls

all_data			example_content
basicstats_clean_tax.ipynb	example_content_copy


move back to example_content directory

In [7]:
cd example_content

locate all files ending with .gz in all subfolders within the directory. The `*` character means that any other characters can preceed .gz. The `mindepth` command specifies to perform commands that follow at the subdirectory level (1=root). The empty `{}` allows all files meeting the criteria to be moved.  The `print` command allows user to monitor files

In [8]:
find . -mindepth 2 -type f -name '*.gz' -print -exec mv {} ../all_data \;

./ERR1424901/ERR1424901_2.fastq.gz
./ERR1424901/ERR1424901_1.fastq.gz
./ERR1424900/ERR1424900_1.fastq.gz
./ERR1424900/ERR1424900_2.fastq.gz
./ERR1424899/ERR1424899_2.fastq.gz
./ERR1424899/ERR1424899_1.fastq.gz
./ERR1424902/ERR1424902_1.fastq.gz
./ERR1424902/ERR1424902_2.fastq.gz
./ERR1424903/ERR1424903_2.fastq.gz
./ERR1424903/ERR1424903_1.fastq.gz


move into the `all_data` subdirectory to check that all the `.fastq.gz` files have moved.

In [9]:
cd ../all_data

In [10]:
ls

ERR1424899_1.fastq.gz	ERR1424901_1.fastq.gz	ERR1424903_1.fastq.gz
ERR1424899_2.fastq.gz	ERR1424901_2.fastq.gz	ERR1424903_2.fastq.gz
ERR1424900_1.fastq.gz	ERR1424902_1.fastq.gz
ERR1424900_2.fastq.gz	ERR1424902_2.fastq.gz


Now, we delete the original example_content directory, which is empty.

In [11]:
rm -r ../example_content

First, I am going to trim reads using the tool BBDUK from bbtools to avoid adapter contamination. I will start with one file to demonstrate syntax. 

We call the tool bbduk within bbtools by `bbduk.sh`. Then, provide input file and output directory/name. The reference library used for trimming is adapters, which contains all illumina adapter sequences. `ktrim` directions determine whether the 3' (right) or 5' (left) adapters are trimmed. In this case, we are setting it to trim the 3' adapter. 

In [12]:
mkdir ../trimmed_reads

In [13]:
bbduk.sh in=ERR1424899_1.fastq.gz out=../trimmed_reads/trimmed_ERR1424899_1.fastq.gz ref=adapters ktrim=r

/usr/local/Cellar/bbtools/38.94/libexec//calcmem.sh: line 75: [: -v: unary operator expected
Max memory cannot be determined.  Attempting to use 1400 MB.
If this fails, please add the -Xmx flag (e.g. -Xmx24g) to your command, 
or run this program qsubbed or from a qlogin session on Genepool, or set ulimit to an appropriate value.
java -ea -Xmx1400m -Xms1400m -cp /usr/local/Cellar/bbtools/38.94/libexec/current/ jgi.BBDuk in=ERR1424899_1.fastq.gz out=../trimmed_reads/trimmed_ERR1424899_1.fastq.gz ref=adapters ktrim=r
Executing jgi.BBDuk [in=ERR1424899_1.fastq.gz, out=../trimmed_reads/trimmed_ERR1424899_1.fastq.gz, ref=adapters, ktrim=r]
Version 38.94

0.030 seconds.
Initial:
Memory: max=1468m, total=1468m, free=1439m, used=29m

Added 2970 kmers; time: 	0.060 seconds.
Memory: max=1468m, total=1468m, free=1435m, used=33m

Input is being processed as unpaired
Started output streams:	0.065 seconds.
Processing time:   		24.663 seconds.

Input:                  	2708070 reads 		470915036 bases

Now that the reads have been trimmed, let's try to do a simple classification of the 16S genes using the package Kraken2. This was Homebrew installed on my machine. We will redirect raw outputs to `null`

In [15]:
cd ..

In [16]:
mkdir reports

In [25]:
mkdir text

Before running Kraken2, we need to define the path of the taxonomy databases

In [20]:
export KRAKEN2_DB_PATH='/Users/ashley/Applications/kraken2-2.0.9-beta/'

We will run kraken2 taxonomy (using the SILVA 16S database) on the trimmed file, and produce a report. Confidence (0-1) will be set to 0.5. To skip producing default text output, follow input file name with `> /dev/null` . If skipping text output, do not enter `output` information

In [28]:
/Users/ashley/Applications/kraken2-2.0.9-beta/kraken2 --db silva --confidence 0.5 --output text/kraken2_text_trimmed_ERR1424899_1.tsv --report reports/kraken2_report_trimmed_ERR1424899_1.tsv trimmed_reads/trimmed_ERR1424899_1.fastq.gz

Loading database information... done.
2707558 sequences (469.54 Mbp) processed in 43.434s (3740.3 Kseq/m, 648.63 Mbp/m).
  7097 sequences classified (0.26%)
  2700461 sequences unclassified (99.74%)


The report has a particular format. Let's take a look at the first few lines.The first column is the percentage of sequence fragments covered by the clade root, the second column is the number of sequence fragments covered by the clade root, the 3rd column is number of sequence fragments assigned directly to the taxon, the 4th column is the taxonomic level, the 5th column is the NCBI taxonomic ID, and the final column is the taxonomy (scientific name)

In [29]:
head reports/kraken2_report_trimmed_ERR1424899_1.tsv

 99.74	2700461	2700461	U	0	unclassified
  0.26	7097	203	R	1	root
  0.20	5385	1486	D	4	  Eukaryota
  0.14	3784	0	D1	46848	    Amorphea
  0.14	3784	0	D2	46957	      Obazoa
  0.14	3784	3	D3	46958	        Opisthokonta
  0.14	3781	0	D	46959	          Holozoa
  0.14	3781	1	D1	46960	            Choanozoa
  0.14	3780	0	D2	47021	              Metazoa
  0.14	3780	0	K	47022	                Animalia


next, let's take a look at the text output, which also has a particular format. The first column indicates whether the sequence was classified or not, the second column is the sequence ID within the fastq file, the 3rd column is the taxonomy ID assigned to the sequence by Kraken2 (0 if unclassified), the 4th column is the sequence length in bp, and the last column is the kmer map. For example- 0:66 means 66 kmers were mapped to unclassified. 

In [30]:
head text/kraken2_text_trimmed_ERR1424899_1.tsv

U	ERR1424899.601	0	100	0:66
U	ERR1424899.602	0	75	0:41
U	ERR1424899.603	0	220	0:186
U	ERR1424899.604	0	285	0:251
U	ERR1424899.605	0	296	0:262
U	ERR1424899.606	0	215	0:181
U	ERR1424899.607	0	95	0:61
U	ERR1424899.608	0	215	0:181
U	ERR1424899.609	0	130	0:96
U	ERR1424899.610	0	85	0:51


Great! now we know how to trim and classify a fastq read file. But what if (as is the case most of the time), it is paired-end? This simply requires a couple of extra arguments in bbduk and kraken2. For bbduk, I am using the recommended settings for trimming by the developers. `k` specifies the kmer length in bp. `mink ` is the minimum allowable kmer length at the end of the sequence, `hdist` is allowable mismatch, `tpe` is trimmed paired end (or make both trimmed reads the same length), `tbo` is trim by overlap, and `ow=t` allows existing files to be overwritten

bbduk.sh in=all_data/ERR1424899_1.fastq.gz in2=all_data/ERR1424899_2.fastq.gz out=trimmed_reads/trimmed_ERR1424899_1.fastq.gz out2=trimmed_reads/trimmed_ERR1424899_2.fastq.gz ref=adapters ktrim=r k=23 mink=11 hdist=1 tpe tbo ow=t

Then run kraken2 on the paired-end data

In [33]:
/Users/ashley/Applications/kraken2-2.0.9-beta/kraken2 --db silva --confidence 0.5 --output text/kraken2_text_trimmed_ERR1424899_paired.tsv --report reports/kraken2_report_trimmed_ERR1424899_paired.tsv --paired trimmed_reads/trimmed_ERR1424899_1.fastq.gz trimmed_reads/trimmed_ERR1424899_2.fastq.gz

Loading database information... done.
2707385 sequences (930.91 Mbp) processed in 82.742s (1963.3 Kseq/m, 675.05 Mbp/m).
  10538 sequences classified (0.39%)
  2696847 sequences unclassified (99.61%)


In [34]:
head reports/kraken2_report_trimmed_ERR1424899_paired.tsv

 99.61	2696847	2696847	U	0	unclassified
  0.39	10538	205	R	1	root
  0.32	8597	1746	D	4	  Eukaryota
  0.25	6743	0	D1	46848	    Amorphea
  0.25	6743	0	D2	46957	      Obazoa
  0.25	6743	2	D3	46958	        Opisthokonta
  0.25	6740	0	D	46959	          Holozoa
  0.25	6740	2	D1	46960	            Choanozoa
  0.25	6738	0	D2	47021	              Metazoa
  0.25	6738	0	K	47022	                Animalia


Note that the output text is slightly different because it is from paired reads. So, in column 4 the format is bp forward read|bp reverse read. the `|` character is used to indicate read types in column 5 also.

In [35]:
head text/kraken2_text_trimmed_ERR1424899_paired.tsv

U	ERR1424899.1	0	145|136	0:110 2375:1 |:| 0:102
U	ERR1424899.2	0	145|193	0:111 |:| 0:159
U	ERR1424899.3	0	143|139	0:109 |:| 0:105
U	ERR1424899.4	0	115|193	0:81 |:| 0:159
U	ERR1424899.5	0	185|182	0:151 |:| 0:148
U	ERR1424899.6	0	203|164	0:121 26859:2 0:46 |:| 0:130
U	ERR1424899.7	0	100|174	0:66 |:| 0:126 26856:2 0:12
U	ERR1424899.8	0	293|182	0:259 |:| 0:36 5610:2 0:110
U	ERR1424899.9	0	120|120	0:86 |:| 0:86
U	ERR1424899.10	0	263|203	0:229 |:| 0:36 26856:2 0:39 26856:2 0:90


Now that we know what to do for one pair of reads, let's loop through all of the read pairs. This loop will set the variable `prefix` to each unique read set name

In [36]:
cd all_data

In [39]:
for prefix in `ls *.gz | cut -f1 -d'_' | sort -u`; do
echo $prefix
read1=( ${prefix}*_1.fastq.gz ) 
read2=( ${prefix}*_2.fastq.gz )

bbduk.sh in=${read1} in2=${read2} out=../trimmed_reads/trimmed_${read1} out2=../trimmed_reads/trimmed_${read2} ref=adapters ktrim=r k=23 mink=11 hdist=1 tbo ow=t
/Users/ashley/Applications/kraken2-2.0.9-beta/kraken2 --db silva --confidence 0.5 --output ../text/kraken2_text_trimmed_${prefix}.tsv --report ../reports/kraken2_report_trimmed_${prefix}.tsv --paired ../trimmed_reads/trimmed_${read1} ../trimmed_reads/trimmed_${read2}
done

ERR1424899
/usr/local/Cellar/bbtools/38.94/libexec//calcmem.sh: line 75: [: -v: unary operator expected
Max memory cannot be determined.  Attempting to use 1400 MB.
If this fails, please add the -Xmx flag (e.g. -Xmx24g) to your command, 
or run this program qsubbed or from a qlogin session on Genepool, or set ulimit to an appropriate value.
java -ea -Xmx1400m -Xms1400m -cp /usr/local/Cellar/bbtools/38.94/libexec/current/ jgi.BBDuk in=ERR1424899_1.fastq.gz in2=ERR1424899_2.fastq.gz out=../trimmed_reads/trimmed_ERR1424899_1.fastq.gz out2=../trimmed_reads/trimmed_ERR1424899_2.fastq.gz ref=adapters ktrim=r k=23 mink=11 hdist=1 tbo ow=t
Executing jgi.BBDuk [in=ERR1424899_1.fastq.gz, in2=ERR1424899_2.fastq.gz, out=../trimmed_reads/trimmed_ERR1424899_1.fastq.gz, out2=../trimmed_reads/trimmed_ERR1424899_2.fastq.gz, ref=adapters, ktrim=r, k=23, mink=11, hdist=1, tbo, ow=t]
Version 38.94

maskMiddle was disabled because useShortKmers=true
0.040 seconds.
Initial:
Memory: max=1468m, total=1468m, f

Now we will use Krona to visualize our results. Because this was conda installed, into my metagenome conda environment, I will need to activate the conda environment first. 

to list all the packages in the environment:

In [2]:
conda list --name metagenome

# packages in environment at /Users/ashley/miniconda3/envs/metagenome:
#
# Name                    Version                   Build  Channel
c-ares                    1.17.1               h9ed2024_0  
ca-certificates           2021.10.26           hecd8cb5_2  
curl                      7.80.0               hca72f7f_0  
krb5                      1.19.2               hcd88c3b_0  
krona                     2.8.1           pl5262hdfd78af_0    bioconda
libcurl                   7.80.0               h6dfd666_0  
libcxx                    12.0.0               h2f01273_0  
libedit                   3.1.20210910         hca72f7f_0  
libev                     4.33                 h9ed2024_1  
libnghttp2                1.46.0               ha29bfda_0  
libssh2                   1.9.0                ha12b0ac_1  
ncurses                   6.3                  hca72f7f_2  
openssl                   1.1.1l               h9ed2024_0  
perl                      5.26.2               h4e221da_0  
zlib     